In [39]:
import os
from langchain import hub
from langchain.agents import create_react_agent, AgentExecutor
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.tools import tool
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, PromptTemplate
from dotenv import load_dotenv
load_dotenv()

True

In [40]:
# 1) Define tools (here: Tavily web search)
tools = [TavilySearchResults(max_results=1)]


# 2) Load the standard ReAct prompt from the LangChain Hub
prompt_template = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}"""

prompt = PromptTemplate.from_template(prompt_template)

In [43]:
# 3) Choose an LLM

os.environ.setdefault("no_proxy", "127.0.0.1,localhost")
os.environ.setdefault("HTTPX_NO_PROXY", "127.0.0.1,localhost")
OLLAMA_URL = os.getenv("OLLAMA_URL", "http://127.0.0.1:11434")

model_name = "gemma3" #"llama3.2:3b"
max_tokens = 100
llm = ChatOllama(model=model_name, base_url=OLLAMA_URL, num_predict=max_tokens, temperature=0.5)

In [44]:
# 4) Create the ReAct agent

agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)

# 5) Wrap it in an AgentExecutor
agent_executor  = AgentExecutor(agent=agent, tools=tools, verbose=True)

# 6) Run the agent
result = agent_executor.invoke({"input": "What is LangChain?"})
print(result["output"])



> Entering new AgentExecutor chain...
I need to understand what LangChain is. I should use the search engine to get a comprehensive overview.
Action: tavily_search_results_json
Action Input: "LangChain framework"[{'title': 'Introduction to LangChain - GeeksforGeeks', 'url': 'https://www.geeksforgeeks.org/artificial-intelligence/introduction-to-langchain/', 'content': 'The LangChain framework is a great interface to develop interesting AI-powered applications and from personal assistants to prompt management as well as automating tasks. So, keep learning and keep developing powerful applications.', 'score': 0.99991107}]I have found a good overview of LangChain from GeeksforGeeks. It describes LangChain as a framework for developing AI-powered applications, including personal assistants and prompt management.
Thought: I now know the final answer
Final Answer: LangChain is a framework designed to help developers build AI-powered applications, such as personal assistants and prompt manag

In [ ]:
# Test Tavily
from tavily import TavilyClient
client = TavilyClient()
response = client.search(
    query="Agentic AI"
)
print(response)

{'query': 'Agentic AI', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.salesforce.com/agentforce/what-is-agentic-ai/', 'title': 'What is Agentic AI?', 'content': '# What is Agentic AI? Agentic AI is an intelligent system that can act autonomously, reason through multi-step problems, and adapt its actions in real-time to achieve a specific business goal with minimal human supervision. This platform includes a suite of tools and services to help AI agents learn, adapt, and collaborate to efficiently handle complex, dynamic tasks. With this framework, businesses can customize AI agents to meet specific needs, whether those needs involve automating repetitive tasks, enhancing customer service, or driving strategic decision-making. ### What agentic AI is. ### What agentic AI is not. By automating complex, multi-step tasks and making decisions independently, AI agents significantly speed up operational processes. With real-time decision-making and